In [1]:
import numpy as np
import pandas as pd
from importlib import reload

In [2]:
# define constants
TRACE_FILE_NAME = 'data/tencent_game_na.pcapng' # replace with your raw trace .pcapng/.pcap file name
TRACE_TCP_PACKET_FEATURE_FILE_NAME = 'data/tencent_game_na_tcp_pkt.csv' # replace with your favorite tcp packet feature .csv file name
TRACE_UDP_PACKET_FEATURE_FILE_NAME = 'data/tencent_game_na_udp_pkt.csv' # replace with your favorite udp packet feature .csv file name
TRACE_PACKET_FEATURE_FILE_NAME = 'data/tencent_game_na_pkt.csv' # replace with your favorite packet feature .csv file name
# LOCAL_IP = '172.16.26.207' # your local ip

## Extract TCP packets

In [3]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.tcp_generate(TRACE_FILE_NAME,TRACE_TCP_PACKET_FEATURE_FILE_NAME)

Conversion done
CPU times: user 2.6 ms, sys: 6.17 ms, total: 8.77 ms
Wall time: 9.21 s


In [4]:
# read in packet feature csv file and do some transformation
import ipaddress
tcp_pkt_feature_df = pd.read_csv(TRACE_TCP_PACKET_FEATURE_FILE_NAME)
filterer = tcp_pkt_feature_df.apply(lambda row:(not pd.isnull(row['ip.src']) and ipaddress.IPv4Address(row['ip.src']).is_global) or (not pd.isnull(row['ip.dst']) and ipaddress.IPv4Address(row['ip.dst']).is_global),axis=1)
tcp_pkt_feature_df = tcp_pkt_feature_df[filterer]
record_num = tcp_pkt_feature_df.shape[0]
tcp_pkt_feature_df['remote_ip'] = tcp_pkt_feature_df.apply(lambda row:row['ip.dst'] if ipaddress.IPv4Address(row['ip.dst']).is_global else row['ip.src'],axis=1) if record_num > 0 else None
tcp_pkt_feature_df['remote_ip2num'] = tcp_pkt_feature_df.apply(lambda row:int(ipaddress.IPv4Address(row['remote_ip'])),axis=1) if record_num > 0 else None
tcp_pkt_feature_df['protocol'] = 'tcp' if record_num > 0 else None
tcp_pkt_feature_df['is_tcp'] = 1 if record_num > 0 else None
tcp_pkt_feature_df['is_udp'] = 0 if record_num > 0 else None
tcp_pkt_feature_df.rename(columns={'tcp.len':'pkt_len'},inplace=True)

In [5]:
# view the shape of the dataset: (number of records, number of features)
tcp_pkt_feature_df.shape

(52802, 17)

In [6]:
# view the data types for each feature
tcp_pkt_feature_df.dtypes

ip.src                  object
ip.dst                  object
tcp.srcport              int64
tcp.dstport              int64
pkt_len                  int64
frame.time_relative    float64
tcp.seq                  int64
tcp.ack                  int64
tcp.flags.ack            int64
tcp.flags.syn            int64
tcp.flags.fin            int64
tcp.stream               int64
remote_ip               object
remote_ip2num            int64
protocol                object
is_tcp                   int64
is_udp                   int64
dtype: object

In [7]:
# view the statistical features of each numerical feature
tcp_pkt_feature_df.describe()

,tcp.srcport,tcp.dstport,pkt_len,frame.time_relative,tcp.seq,tcp.ack,tcp.flags.ack,tcp.flags.syn,tcp.flags.fin,tcp.stream,remote_ip2num,is_tcp,is_udp
count,52802.000000,52802.000000,52802.000000,52802.000000,5.280200e+04,5.280200e+04,52802.000000,52802.000000,52802.000000,52802.000000,5.280200e+04,52802.0,52802.0
mean,17299.196621,26272.557384,762.424075,65.674473,3.093605e+05,1.769940e+05,0.990644,0.007140,0.005738,74.305443,1.530230e+09,1.0,0.0
std,20903.021846,20906.418760,668.026356,184.472434,4.038899e+05,3.381204e+05,0.096272,0.084196,0.075535,28.333318,8.704069e+08,0.0,0.0
min,80.000000,80.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,2.880297e+08,1.0,0.0
25%,443.000000,443.000000,0.000000,22.638198,7.490000e+02,5.080000e+02,1.000000,0.000000,0.000000,58.000000,4.004113e+08,1.0,0.0
50%,443.000000,36444.000000,1368.000000,24.453401,1.038070e+05,9.900000e+02,1.000000,0.000000,0.000000,68.000000,2.163119e+09,1.0,0.0
75%,40288.000000,45140.000000,1368.000000,27.152311,5.381900e+05,2.005992e+05,1.000000,0.000000,0.000000,79.000000,2.163119e+09,1.0,0.0
max,63704.000000,63704.000000,1380.000000,1338.640061,1.883614e+06,1.883638e+06,1.000000,1.000000,1.000000,232.000000,3.736744e+09,1.0,0.0


In [8]:
# view the first 5 records
tcp_pkt_feature_df.head()

,ip.src,ip.dst,tcp.srcport,tcp.dstport,pkt_len,frame.time_relative,tcp.seq,tcp.ack,tcp.flags.ack,tcp.flags.syn,tcp.flags.fin,tcp.stream,remote_ip,remote_ip2num,protocol,is_tcp,is_udp
0,128.238.147.112,17.248.135.181,63660,443,1368,0.000000,1,1,1,0,0,0,17.248.135.181,301500341,tcp,1,0
1,128.238.147.112,17.248.135.181,63660,443,101,0.000001,1369,1,1,0,0,0,17.248.135.181,301500341,tcp,1,0
2,128.238.147.112,17.248.135.181,63660,443,1368,0.000274,1470,1,1,0,0,0,17.248.135.181,301500341,tcp,1,0
3,17.248.135.181,128.238.147.112,443,63660,0,0.001493,1,1470,1,0,0,0,128.238.147.112,2163118960,tcp,1,0
4,128.238.147.112,17.248.135.181,63660,443,824,0.001524,2838,1,1,0,0,0,17.248.135.181,301500341,tcp,1,0


## Extract UDP packets

In [9]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(TRACE_FILE_NAME,TRACE_UDP_PACKET_FEATURE_FILE_NAME,True)

Conversion done


No error


CPU times: user 4.27 ms, sys: 5.45 ms, total: 9.73 ms
Wall time: 7.6 s


In [10]:
udp_pkt_feature_df = pd.read_csv(TRACE_UDP_PACKET_FEATURE_FILE_NAME)
udp_pkt_feature_df

,ip.src,ip.dst,udp.srcport,udp.dstport,udp.length,frame.time_relative,udp.stream
0,128.238.147.3,224.0.0.2,1985,1985,28,0.097179,0
1,128.238.147.2,224.0.0.2,1985,1985,28,0.100842,1
2,128.238.147.145,128.238.147.255,37481,1534,16,0.574101,2
3,128.238.147.3,224.0.0.2,1985,1985,28,3.096920,0
4,128.238.147.2,224.0.0.2,1985,1985,28,3.101557,1
5,128.238.147.137,255.255.255.255,17500,17500,152,4.347565,3
6,128.238.147.137,128.238.147.255,17500,17500,152,4.347699,4
7,128.238.147.112,128.238.1.69,64703,53,40,4.791367,5
8,128.238.147.112,128.238.1.69,51917,53,45,4.791480,6
9,128.238.147.112,128.238.1.69,61129,53,49,4.791661,7


In [11]:
# read in packet feature csv file and do some transformation
import ipaddress
def filter_illegal(row):
    try:
        ipaddress.IPv4Address(row['ip.src'])
        ipaddress.IPv4Address(row['ip.dst'])
        return (not pd.isnull(row['ip.src']) and ipaddress.IPv4Address(row['ip.src']).is_global) or (not pd.isnull(row['ip.dst']) and ipaddress.IPv4Address(row['ip.dst']).is_global)
    except ValueError as e:
        print(e)
        return False                                                                                
                                                                                                
udp_pkt_feature_df = pd.read_csv(TRACE_UDP_PACKET_FEATURE_FILE_NAME)
filterer = udp_pkt_feature_df.apply(filter_illegal,axis=1)
udp_pkt_feature_df = udp_pkt_feature_df[filterer]
record_num = udp_pkt_feature_df.shape[0]
udp_pkt_feature_df['remote_ip'] = udp_pkt_feature_df.apply(lambda row:row['ip.dst'] if ipaddress.IPv4Address(row['ip.dst']).is_global else row['ip.src'],axis=1) if record_num > 0 else None
udp_pkt_feature_df['remote_ip2num'] = udp_pkt_feature_df.apply(lambda row:int(ipaddress.IPv4Address(row['remote_ip'])),axis=1) if record_num > 0 else None
udp_pkt_feature_df['protocol'] = 'udp' if record_num > 0 else None
udp_pkt_feature_df['is_tcp'] = 0 if record_num > 0 else None
udp_pkt_feature_df['is_udp'] = 1 if record_num > 0 else None
udp_pkt_feature_df.rename(columns={'udp.length':'pkt_len'},inplace=True)

Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
Expected 4 octets in 'nan'
E

In [12]:
# view the shape of the dataset: (number of records, number of features)
udp_pkt_feature_df.shape

(43819, 12)

In [13]:
# view the data types for each feature
udp_pkt_feature_df.dtypes

ip.src                  object
ip.dst                  object
udp.srcport              int64
udp.dstport              int64
pkt_len                  int64
frame.time_relative    float64
udp.stream               int64
remote_ip               object
remote_ip2num            int64
protocol                object
is_tcp                   int64
is_udp                   int64
dtype: object

In [14]:
# view the statistical features of each numerical feature
udp_pkt_feature_df.describe()

,udp.srcport,udp.dstport,pkt_len,frame.time_relative,udp.stream,remote_ip2num,is_tcp,is_udp
count,43819.000000,43819.000000,43819.000000,43819.000000,43819.000000,4.381900e+04,43819.0,43819.0
mean,25193.044866,22211.606997,173.944271,716.771043,83.376252,2.550868e+09,0.0,1.0
std,17599.801570,17467.249729,115.121193,338.969400,15.466526,3.987631e+08,0.0,0.0
min,53.000000,53.000000,16.000000,0.097179,0.000000,3.017985e+08,0.0,1.0
25%,6651.000000,6651.000000,102.000000,429.984005,85.000000,2.163119e+09,0.0,1.0
50%,41764.000000,6651.000000,102.000000,724.518986,85.000000,2.838481e+09,0.0,1.0
75%,41764.000000,41764.000000,294.000000,1002.336581,85.000000,2.838481e+09,0.0,1.0
max,65437.000000,65437.000000,532.000000,1338.676886,214.000000,4.026532e+09,0.0,1.0


In [15]:
# view the first 5 records
udp_pkt_feature_df.head()

,ip.src,ip.dst,udp.srcport,udp.dstport,pkt_len,frame.time_relative,udp.stream,remote_ip,remote_ip2num,protocol,is_tcp,is_udp
0,128.238.147.3,224.0.0.2,1985,1985,28,0.097179,0,224.0.0.2,3758096386,udp,0,1
1,128.238.147.2,224.0.0.2,1985,1985,28,0.100842,1,224.0.0.2,3758096386,udp,0,1
2,128.238.147.145,128.238.147.255,37481,1534,16,0.574101,2,128.238.147.255,2163119103,udp,0,1
3,128.238.147.3,224.0.0.2,1985,1985,28,3.096920,0,224.0.0.2,3758096386,udp,0,1
4,128.238.147.2,224.0.0.2,1985,1985,28,3.101557,1,224.0.0.2,3758096386,udp,0,1


## Combine TCP with UDP packets

In [16]:
# combine dataframes
pkt_feature_df = tcp_pkt_feature_df[['remote_ip2num','is_tcp','is_udp','pkt_len']].append(udp_pkt_feature_df[['remote_ip2num','is_tcp','is_udp','pkt_len']],ignore_index=True)

In [17]:
# shape
pkt_feature_df.shape

(96621, 4)

In [18]:
###### column types
pkt_feature_df.dtypes

remote_ip2num    int64
is_tcp           int64
is_udp           int64
pkt_len          int64
dtype: object

In [19]:
# describe
pkt_feature_df.describe()

,remote_ip2num,is_tcp,is_udp,pkt_len
count,9.662100e+04,96621.000000,96621.000000,96621.000000
mean,1.993104e+09,0.546486,0.453514,495.540100
std,8.627334e+08,0.497837,0.497837,579.406738
min,2.880297e+08,0.000000,0.000000,0.000000
25%,2.163119e+09,0.000000,0.000000,32.000000
50%,2.163119e+09,1.000000,0.000000,102.000000
75%,2.838481e+09,1.000000,1.000000,1368.000000
max,4.026532e+09,1.000000,1.000000,1380.000000


In [20]:
# head 5 records
pkt_feature_df.head()

,remote_ip2num,is_tcp,is_udp,pkt_len
0,301500341,1,0,1368
1,301500341,1,0,101
2,301500341,1,0,1368
3,2163118960,1,0,0
4,301500341,1,0,824


In [21]:
# write to csv
pkt_feature_df.to_csv(TRACE_PACKET_FEATURE_FILE_NAME, index=False)